In [ ]:
from __future__ import unicode_literals, print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.utils import data
import os
import itertools
import re
import random
import time
from torch.autograd import Variable
from scipy.signal import tukey
from torch.utils import data
from statsmodels.tsa.arima_model import ARIMA
#import torch.utils.data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import warnings
warnings.filterwarnings("ignore")

## Best Performing Parameters

In [ ]:
def find_best_params(directory):
    pts = os.listdir(directory)
    min_loss = 1000
    best_params = 0 
    for i in range(len(pts)):
        try:
            item = torch.load(directory + "/" + pts[i])
            loss = item["min_valid_loss"]
            if loss <= min_loss:
                min_loss = loss
                best_params = item
        except:
            continue
    return min_loss, best_params 

In [ ]:
folders = [folder for folder in os.listdir(os.getcwd()) if 'ipynb' not in folder]
min_loss = {}
best_params = {}
for fold in folders:
    min_loss[fold], best_params[fold] = find_best_params(fold)

In [ ]:
param_names = ['learning_rate', 'dropout_rate', 'num_layers', 'hidden_dim']
print([(param, best_params['lstms_at'][param]) for param in param_names])
print(('min_valid_loss', min_loss['lstms_at']))

In [ ]:
param_names = ['learning_rate', 'dropout_rate', 'num_layers', 'hidden_dim']
print([(param, best_params['lstms_attention_at'][param]) for param in param_names])
print(('min_valid_loss', min_loss['lstms_attention_at']))

In [ ]:
param_names = ['learning_rate', 'dropout_rate', 'encode_num_layers', 'decode_num_layers', 'hidden_dim', 'kernel_size']
print([(param, best_params['tcn_at'][param]) for param in param_names])
print(('min_valid_loss', min_loss['tcn_at']))

In [ ]:
param_names = ['learning_rate', 'dropout_rate', 'num_layers', 'd_model', 'd_ff', 'h']
print([(param, best_params['transformer_at'][param]) for param in param_names])
print(('min_valid_loss', min_loss['transformer_at']))

In [ ]:
param_names = ['learning_rate', 'dropout_rate', 'feat_dim', 'decode_num_layers', 'hidden_dim', 'kernel_size', 'N', 'num_map_layers']
print([(param, best_params['pyramid_at'][param]) for param in param_names])
print(('min_valid_loss', min_loss['pyramid_at']))

In [ ]:
param_names = ['learning_rate', 'dropout_rate', 'num_layers', 'hidden_dim']
print([(param, best_params['nn_at'][param]) for param in param_names])
print(('min_valid_loss', min_loss['nn_at']))

## Varying Forecasting Horizons

In [ ]:
def vary_horizon(directory):
    # CNN
    lst = os.listdir(directory)
    if len(lst)%10 != 0:
        lst = sorted(lst)[1:]
    else:
        lst = sorted(lst)
    step = int(len(lst)/10)
    horizon_lst = [torch.load(directory + "/" + lst[i])["min_valid_loss"] for i in range(len(lst))]
    horizon_lst = [np.min(horizon_lst[i-step:i]) for i in range(step, len(lst) + step, step) ]
    return sorted(horizon_lst)

In [ ]:
#horizon_cnn_at = vary_horizon("CNN_AT_horizon")
1.34, 0.71662, 0.7795, 0.69257, 0.58275

In [ ]:
horizon_cnn_at[-1]

In [ ]:
horizon_cnn_rt = [0.27598, 0.3315, 0.36877, 0.40756, 0.45133, 0.47069, 0.49381, 0.53656 , 0.54284, 0.58275]
#horizon_cnn_at = vary_horizon("CNN_AT_horizon")
horizon_cnn_at = vary_horizon("CNN_AT_horizon")
horizon_lstm = vary_horizon("LSTM_AT_horizon")
horizon_trans = vary_horizon("Transformer_AT_horizon")
horizon_pyramid = vary_horizon("Pyramid_RT_horizon")
horizon_arima = [1.3208365121960388,
                 1.3163297457108272,
                 1.310415726350439,
                 1.306577285185987,
                 1.3068774655950486,
                 1.3102307160871303,
                 1.315505526815925,
                 1.3220386706471747,
                 1.3306421697079553,
                 1.3413983816490263]


In [ ]:
horizon_pyramid[-1]

In [ ]:
horizon_cnn_rt[-1]

In [ ]:
x = list(range(3,31,3))
plt.plot(x, horizon_cnn_rt, label = "CNN_RT", linewidth = 3)
plt.plot(x, horizon_cnn_at, label = "CNN_AT", linewidth = 3)
#plt.plot(x, horizon_arima, label = "ARIMA_AT", linewidth = 3)
plt.plot(x, horizon_lstm, label = "LSTM_AT", linewidth = 3)
plt.plot(x, horizon_trans, label = "Transformer_AT", linewidth = 3)
plt.plot(x, horizon_pyramid, label = "Pyramid_RT", linewidth = 3)
plt.xlabel("Prediction Steps", fontsize=12)
plt.ylabel("RMSE", fontsize=12)
plt.legend(loc = 2)
#plt.ylim(0,1.5)
plt.title("Real Aortic Pressure Data",fontsize=15)
#plt.savefig("RMSEs_horizon.png", dpi = 400)
plt.show()

## Visualization

In [ ]:
X = torch.load("/home/rui/KDD/Data/AT_X.pt")[8000:,:,0]

In [ ]:
def best_model(directory):
    lst = os.listdir(directory)
    lst = sorted(lst)[1:]
    lst = [lst[k] for k in range(len(lst)) if "30" in lst[k]]
    best_rmse = 100
    best_model = 0
    for i in range(len(lst)):
        item = torch.load(directory + "/" + lst[i])
        if item["min_valid_loss"]  < best_rmse:
            best_rmse = item["min_valid_loss"]
            best_model = item
    return best_model

In [ ]:
#best_model_cnn_rt = best_model("CNN_AT_horizon")
best_model_cnn_at = best_model("CNN_AT_horizon")
best_model_lstm = best_model("LSTM_AT_horizon")
best_model_trans = best_model("Transformer_AT_horizon")
best_model_pyramid = best_model("Pyramid_RT_horizon")

In [ ]:
std = 19.481796
avg = 84.305374
lstm_preds = best_model_lstm["preds"]*std+avg
cnn_preds_at = best_model_cnn_at["preds"]*std+avg
cnn_preds_rt = torch.load("CNN_RT_horizon/genz-30-2.pt")["preds"]*std+avg
pyramid_preds = best_model_pyramid["preds"]*std+avg
trans_preds = best_model_trans["preds"].reshape(2000,30)*std+avg
# ARIMA
arima = np.load("ARIMA_AT_preds.npy")*std+avg
true = np.load("trues_AT.npy")*std+avg

In [ ]:
def smooth(lst):
    lst = list([np.mean(lst[i:i+3]) for i in range(0, len(lst))])
    return np.array(lst)

In [ ]:
index = [350, 360, 620, 940, 970, 1800, 330, 1980, 1870, 1720, 1450, 1290]

In [ ]:
pos = list(range(31,61))
pos2 = list(range(1,61))
trueline = smooth(np.concatenate([X[index[count]], true[index[count]]]))
plt.plot(pos, smooth(trans_preds[index[count]]), label = "Transformer_AT")
plt.plot(pos, smooth(lstm_preds[:,index[count]]), label = "LSTM_AT")
plt.plot(pos, smooth(cnn_preds_at[index[count]]), label = "CNN_AT")
plt.plot(pos, smooth(cnn_preds_rt[index[count]]), label = "CNN_RT")
plt.plot(pos, smooth(pyramid_preds[index[count]]), label = "Pyramid_RT")
plt.plot(pos2, trueline, label = "Ground Truth", linestyle= "--", linewidth = 3)
#plt.plot(smooth(true[index[count]]), label = "Ground Truth", linestyle= "--", linewidth = 3)
plt.show()

In [ ]:
plt.figure(figsize=(20,15)) 
#plt.plot(arima[idx]*std+avg, label = "ARIMA")
count = 0
pos = list(range(31,61))
pos2 = list(range(1,61))
#for i in range(1,5):
for count in range(0,12):
    plt.subplot(4, 3, count+1)
    #print(count)
    trueline = smooth(np.concatenate([X[index[count]], true[index[count]]]))
    plt.plot(pos,smooth(trans_preds[index[count]]), label = "Transformer_AT", linewidth = 3)
    plt.plot(pos,smooth(lstm_preds[:,index[count]]), label = "LSTM_AT", linewidth = 3)
    plt.plot(pos,smooth(cnn_preds_at[index[count]]), label = "CNN_AT", linewidth = 3)
    plt.plot(pos,smooth(cnn_preds_rt[index[count]]), label = "CNN_RT", linewidth = 3)
    plt.plot(pos, smooth(pyramid_preds[index[count]]), label = "Pyramid_RT", linewidth = 3)
    plt.plot(pos2,trueline, label = "Ground Truth", linestyle= "--", linewidth = 4, color = "black")
    plt.axvline(x = 30.5, color='black', linestyle = "--")
    count += 1
    if count == 3:
        plt.legend(loc=2, bbox_to_anchor=(1, 1.05), fontsize = 15)

#plt.savefig("Pred_Case.png", dpi = 400)
plt.tight_layout()
plt.text(-40, 78, 'Input Steps + Predition Steps', ha='center', fontsize = 20)
plt.text(-160, 127, 'Aortic Pressure(mmHg)', va='center', rotation='vertical', fontsize = 20)
plt.savefig("preds_real.png", dpi = 600, bbox_inches = "tight")
plt.show()

## RT

In [ ]:
min_loss_cnn_rt, best_params_cnn_rt = Best_Params("Results/RT_CNN")
#min_loss_lstm_rt, best_params_lstm_rt = Best_Params("Results/RT_LSTM")
#min_loss_trans_rt, best_params_trans_rt = Best_Params2("Results/RT_Transformer")

In [ ]:
avg = 83.25376
std = 18.252565

In [ ]:
arima_rt.shape

In [ ]:
arima_rt = np.load("Results/preds_ARIMA_RT.npy")
true_rt = np.load("Results/trues_ARIMA_RT.npy")
cnn_preds_rt = np.concatenate(torch.load("Results/RT_CNN_horizon/genz-60-1.pt")["preds"], axis = 0)

In [ ]:
lstm_preds_rt = np.concatenate(torch.load("Results/RT_LSTM_horizon/genz-60-0.pt")["preds"], axis = 1)

In [ ]:
lstm_preds_rt.shape

In [ ]:
index = []
for i in range(len(cnn_preds_rt)):
    if np.abs(cnn_preds_rt[i][-1] - cnn_preds_rt[i][0]) > 10/std:
        index.append(i)
    
idx = 30
plt.figure(figsize=(15,5)) 
avg = 82.114876
std = 17.022003

#plt.plot(arima_rt[index[idx]]*std+avg, label = "ARIMA")
plt.plot(cnn_preds_rt[index[idx]][:30]*std+avg, label = "CNN")
#plt.plot(lstm_preds_rt[:,index[idx]]*std+avg, label = "LSTM")
plt.plot(true_rt[index[idx]][:30]*std+avg, label = "True", linestyle= "--")
plt.legend(loc=2)
plt.savefig("Case Four.png", dpi = 400)
plt.show()

In [ ]:
list(best_params_lstm_rt.items())[:4]

In [ ]:
def vary_horizon2(directory):
    # CNN
    lst = os.listdir(directory)
    lst = sorted(lst)[1:]
    horizon_lst = []
    for i in range(len(lst)):
        horizon_lst.append(torch.load(directory + "/" + lst[i])["min_valid_loss"])
    return horizon_lst

In [ ]:
horizon_lstm_rt = vary_horizon2("Results/RT_LSTM_horizon")

In [ ]:
horizon_lstm_rt = [0.59899, 0.62331, 0.6054, 0.60684, 0.60, 0.61, 0.612, 0.66, 0.62, 0.60562]

In [ ]:
horizon_cnn_rt = vary_horizon2("Results/RT_CNN_horizon")

In [ ]:
horizon_arima_rt = [0.9629866302044089,
 0.9708720917980342,
 0.9725228657326156,
 0.9725916145001904,
 0.971447765413801,
 0.9701259809733777,
 0.9690435077421782,
 0.9679333772996392,
 0.9666859927240498,
 0.9658913494110664]
x = list(range(6,61,6))

plt.plot(x, horizon_arima_rt, label = "ARIMA", linewidth = 3)
plt.plot(x, horizon_cnn_rt, label = "CNN", linewidth = 3)
plt.plot(x, horizon_lstm_rt, label = "LSTM", linewidth = 3)
plt.legend(loc = 2)
plt.xlabel("Prediction Steps", fontsize=12)
plt.ylabel("RMSE", fontsize=12)

#plt.ylim(0,0.55)
plt.title("RMSEs for varying forecasting horizons",fontsize=15)
plt.savefig("RMSE_horizon_rt.png", dpi = 400)
plt.show()

## Visualize the weight

In [ ]:
weight = torch.load("Results/RT_CNN_horizon/genz-60-1.pt")['model_state_dict']
weight1 = weight['input_layer.0.weight']
weight2 = weight['input_layer.1.weight']

In [ ]:

import torchvision.models as models

def plot_kernels(tensor, num_cols=5):
    num_kernels = tensor.shape[0]
    num_rows = 1+ num_kernels // num_cols
    fig = plt.figure(figsize=(num_cols,num_rows))
    for i in range(tensor.shape[0]):
        ax1 = fig.add_subplot(num_rows,num_cols,i+1)
        ax1.imshow(tensor[i])
        ax1.axis('off')
        ax1.set_xticklabels([])
        ax1.set_yticklabels([])

    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig("weight visualization.png", dpi = 400)
    plt.show()
    

In [ ]:
plot_kernels(-np.abs(weight1[:30,:,:].cpu().detach().numpy()))